# Report Cards

Report Cards is a iPython Nootbook script to generate a text file containing the “report card” of all students.

## Installation


If you do not have the libraries installed on the Google Colab, add and run the commands on first cell in this notebook.


Prerequsite Python library package:
```bash
!pip install pandas
!pip install numpy
!pip install io
```

## Run the script
1) Download courses, marks, students, tests csv file into the folder

2) Download this ipynb file into local folder, upload it into the Google Drive and run it with Google Colab

3) Based on the code instruction to upload files into this script

4) The code will automatically generate the txt file and download the report card into the local system. 




## Extract the data 

In [0]:
# upload csv file to google drive, need to run it in Chrome browser
from google.colab import files
import io
import pandas as pd
import numpy as np

# Upload the files
uploaded_file = files.upload()

# read the uploaded file to data set
course_data = io.BytesIO(uploaded_file['courses.csv'])    

Saving courses.csv to courses (1).csv


In [0]:
# read data file
course_df = pd.read_csv(course_data,  sep = ",")

# create new columns copies
course_df['course_id'] = course_df['id']
course_df['course_name'] = course_df['name']

# remove the columns 
del course_df['id']
del course_df['name']

course_df.head()

,teacher,course_id,course_name
0,Mr. D,1,Biology
1,Mrs. P,2,History
2,Mrs. C,3,Math


In [0]:
# Upload the files
from google.colab import files
import io
uploaded_file = files.upload()

# read the uploaded file to data set
marks_data = io.BytesIO(uploaded_file['marks.csv'])  

Saving marks.csv to marks (1).csv


In [0]:
# read data file
marks_df = pd.read_csv(marks_data,  sep = ",")
marks_df.head()

,test_id,student_id,mark
0,1,1,78
1,2,1,87
2,3,1,95
3,4,1,32
4,5,1,65


In [0]:
from google.colab import files
# Upload the files
uploaded_file = files.upload()

# read the uploaded file to data set
students_data = io.BytesIO(uploaded_file['students.csv'])   

Saving students.csv to students (1).csv


In [0]:
# read data file
students_df = pd.read_csv(students_data,  sep = ",")

# create new columns copies
students_df['student_id'] = students_df['id']
students_df['student_name'] = students_df['name']

del students_df['id']
del students_df['name']

students_df.head()

,student_id,student_name
0,1,A
1,2,B
2,3,C


In [0]:
from google.colab import files
# Upload the files
uploaded_file = files.upload()

# read the uploaded file to data set
tests_data = io.BytesIO(uploaded_file['tests.csv'])   

Saving tests.csv to tests (1).csv


In [0]:
# read data file
tests_df = pd.read_csv(tests_data,  sep = ",")
tests_df['test_id'] = tests_df['id']
del tests_df['id']
tests_df.head()

,course_id,weight,test_id
0,1,10,1
1,1,40,2
2,1,50,3
3,2,40,4
4,2,60,5


## Transfer the data

Merge student and mark dataframe

In [0]:
# merge student and marks dataframe
student_mark_df = pd.merge(students_df, marks_df, on = 'student_id')
student_mark_df.head(10)

,student_id,student_name,test_id,mark
0,1,A,1,78
1,1,A,2,87
2,1,A,3,95
3,1,A,4,32
4,1,A,5,65
5,1,A,6,78
6,1,A,7,40
7,2,B,1,78
8,2,B,2,87
9,2,B,3,15


Merge course and tests dataframe

In [0]:
# merge course and tests dataframe
course_test_df = pd.merge(course_df, tests_df, on = 'course_id')
course_test_df.head(10)

,teacher,course_id,course_name,weight,test_id
0,Mr. D,1,Biology,10,1
1,Mr. D,1,Biology,40,2
2,Mr. D,1,Biology,50,3
3,Mrs. P,2,History,40,4
4,Mrs. P,2,History,60,5
5,Mrs. C,3,Math,90,6
6,Mrs. C,3,Math,10,7


Merge all the dataframe into one main dataframe

In [0]:
# merge all data frame into one 
df = pd.merge(student_mark_df, course_test_df, on = 'test_id')
df.head()

,student_id,student_name,test_id,mark,teacher,course_id,course_name,weight
0,1,A,1,78,Mr. D,1,Biology,10
1,2,B,1,78,Mr. D,1,Biology,10
2,3,C,1,78,Mr. D,1,Biology,10
3,1,A,2,87,Mr. D,1,Biology,40
4,2,B,2,87,Mr. D,1,Biology,40


In [0]:
# calculate weighted grade
df['weight_grade'] = df['mark'] * df['weight'] / 100
df.head()

,student_id,student_name,test_id,mark,teacher,course_id,course_name,weight,weight_grade
0,1,A,1,78,Mr. D,1,Biology,10,7.8
1,2,B,1,78,Mr. D,1,Biology,10,7.8
2,3,C,1,78,Mr. D,1,Biology,10,7.8
3,1,A,2,87,Mr. D,1,Biology,40,34.8
4,2,B,2,87,Mr. D,1,Biology,40,34.8


In [0]:
# calculate final grade
total_df = df.groupby(['student_id','student_name', 'teacher', 'course_name']).agg( 
    final_grade = ('weight_grade', sum)
    )
total_df.head(10)

final_grade
student_id student_name teacher course_name             
1          A             Mrs. C Math                74.2
                         Mrs. P History             51.8
                        Mr. D   Biology             90.1
2          B             Mrs. C Math                74.2
                        Mr. D   Biology             50.1
3          C             Mrs. C Math                74.2
                         Mrs. P History             51.8
                        Mr. D   Biology             90.1

In [0]:
# calculate total average
grade_df = total_df.groupby('student_id')['final_grade'].mean().to_frame()
grade_df.head()

,final_grade
student_id,
1,72.033333
2,62.150000
3,72.033333


In [0]:
# round 2 decimal number and rename the column
grade_df['total_average'] = round(grade_df['final_grade'], 2)
del grade_df['final_grade']
grade_df.head()

,total_average
student_id,
1,72.03
2,62.15
3,72.03


In [0]:
# break the aggregated table into dataframe
total_df = total_df.reset_index()
total_df.head()

,student_id,student_name,teacher,course_name,final_grade
0,1,A,Mrs. C,Math,74.2
1,1,A,Mrs. P,History,51.8
2,1,A,Mr. D,Biology,90.1
3,2,B,Mrs. C,Math,74.2
4,2,B,Mr. D,Biology,50.1


In [0]:
# add total average column into the dataframe
total_grade_df = pd.merge(total_df, grade_df, on='student_id')
total_grade_df.head()

,student_id,student_name,teacher,course_name,final_grade,total_average
0,1,A,Mrs. C,Math,74.2,72.03
1,1,A,Mrs. P,History,51.8,72.03
2,1,A,Mr. D,Biology,90.1,72.03
3,2,B,Mrs. C,Math,74.2,62.15
4,2,B,Mr. D,Biology,50.1,62.15


In [0]:
# display the table in a aggregated table
total_grade_df.groupby(['student_id',	'student_name', 'teacher', 'course_name', 'final_grade', 'total_average']).agg( 
    student_id_count = ('student_id', max)
    )

student_id_count
student_id student_name teacher course_name final_grade total_average                  
1          A             Mrs. C Math        74.2        72.03                         1
                         Mrs. P History     51.8        72.03                         1
                        Mr. D   Biology     90.1        72.03                         1
2          B             Mrs. C Math        74.2        62.15                         2
                        Mr. D   Biology     50.1        62.15                         2
3          C             Mrs. C Math        74.2        72.03                         3
                         Mrs. P History     51.8        72.03                         3
                        Mr. D   Biology     90.1        72.03                         3

## Export the data

In [0]:
from google.colab import files

# set the flag for student grade printing
student_id = 0

# write the output to text file
with open('grade_report.txt', 'w') as grade_file:

  grade_file.write('\n')

  # iterate each line in the dataframe
  for i in range(len(total_grade_df)):

      # if reading the same student, we print out the course info
      if total_grade_df.at[i,'student_id'] == student_id:

        # print course information and grade informaton
        print('    Course: ' + str(total_grade_df.at[i,'course_name']) + ', Teacher: ' + str(total_grade_df.at[i,'teacher']))
        print('    Final Grade:   ' + str(total_grade_df.at[i,'final_grade']) +'%')
        print()
        
        # write into the file
        grade_file.write('    Course: ' + str(total_grade_df.at[i,'course_name']) + ', Teacher: ' + str(total_grade_df.at[i,'teacher']) + '\n')
        grade_file.write('    Final Grade:   ' + str(total_grade_df.at[i,'final_grade']) +'%\n')
        grade_file.write('\n')

      # for new student in the next line
      else:
        # display on the screen to check the result
        print()
        print('Student Id: ' + str(total_grade_df.at[i,'student_id']) + ' name: ' + str(total_grade_df.at[i,'student_name']))
        print('Total Average:     ' + str(total_grade_df.at[i,'total_average']) + '%')
        print()
        print('    Course: ' + str(total_grade_df.at[i,'course_name']) + ', Teacher: ' + str(total_grade_df.at[i,'teacher']))
        print('    Final Grade:   ' + str(total_grade_df.at[i,'final_grade']) +'%')
        print()


        # write into the file
        grade_file.write('\n')
        grade_file.write('Student Id: ' + str(total_grade_df.at[i,'student_id']) + ' name: ' + str(total_grade_df.at[i,'student_name']))
        grade_file.write('Total Average:     ' + str(total_grade_df.at[i,'total_average']) + '%')

        grade_file.write('\n')
        grade_file.write('    Course: ' + str(total_grade_df.at[i,'course_name']) + ', Teacher: ' + str(total_grade_df.at[i,'teacher']) + '\n')
        grade_file.write('    Final Grade:   ' + str(total_grade_df.at[i,'final_grade']) +'%\n')
        grade_file.write('\n')

        # change to next student
        student_id += 1

# download the file in txt format  
files.download('grade_report.txt')



Student Id: 1 name: A
Total Average:     72.03%

    Course: Math, Teacher:  Mrs. C
    Final Grade:   74.2%

    Course: History, Teacher:  Mrs. P
    Final Grade:   51.8%

    Course: Biology, Teacher: Mr. D
    Final Grade:   90.1%


Student Id: 2 name: B
Total Average:     62.15%

    Course: Math, Teacher:  Mrs. C
    Final Grade:   74.2%

    Course: Biology, Teacher: Mr. D
    Final Grade:   50.099999999999994%


Student Id: 3 name: C
Total Average:     72.03%

    Course: Math, Teacher:  Mrs. C
    Final Grade:   74.2%

    Course: History, Teacher:  Mrs. P
    Final Grade:   51.8%

    Course: Biology, Teacher: Mr. D
    Final Grade:   90.1%

